Тест

In [1]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka import EmotionRecognizer

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kozak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-06-05 12:51:15,828 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


In [2]:
#загрузка датафрейма
import pandas as pd
df = pd.read_csv('C:/Users/kozak/OneDrive/Рабочий стол/итмо поступление/НИРМА/Комменты и посты/Парсер/Индентичность/comments_grazhdanka_spb_kalininsky.csv', sep = ';', index_col=0)

In [3]:
df = df.head(100)

In [7]:
df

,name,last_name,date,text,post_id
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0
1370278,Irina,Vladimirovna,2023-01-16,"А кто знает,почему от этой школы шарахались,ка...",1370271.0
1370412,Alya,Dzhafarova,2023-01-17,"Интересно ,а как в нее можно попасть?Если в 3 ...",1370271.0
...,...,...,...,...,...
1370825,Nina,Khryakova,2023-01-19,Очень вкусная и отличная продукция все натурал...,1370426.0
1370547,Vika,Lee,2023-01-18,Не жрать что попало ))),1370468.0
1370537,Vyacheslav,Slivkin,2023-01-17,Есть в основной массе белки и сложные углеводы...,1370468.0
1370529,Vitek,Sokol,2023-01-17,Первитин или аналоги.,1370468.0


Быстрый способо определить эмоцию, используя одну модель. По умолчанию стоит модель HuggingFaceModel.Text.Bert_Large, но можно использовать также HuggingFaceModel.Text.Bert_Tiny, HuggingFaceModel.Text.Bert_Base, HuggingFaceModel.Text.Bert_Tiny2

In [9]:
recognizer = EmotionRecognizer()
df['emotion'] = df['text'].apply(recognizer.recognize_emotion)

In [10]:
df

,name,last_name,date,text,post_id,emotion
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0,neutral
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0,sadness
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0,neutral
1370278,Irina,Vladimirovna,2023-01-16,"А кто знает,почему от этой школы шарахались,ка...",1370271.0,enthusiasm
1370412,Alya,Dzhafarova,2023-01-17,"Интересно ,а как в нее можно попасть?Если в 3 ...",1370271.0,enthusiasm
...,...,...,...,...,...,...
1370825,Nina,Khryakova,2023-01-19,Очень вкусная и отличная продукция все натурал...,1370426.0,happiness
1370547,Vika,Lee,2023-01-18,Не жрать что попало ))),1370468.0,happiness
1370537,Vyacheslav,Slivkin,2023-01-17,Есть в основной массе белки и сложные углеводы...,1370468.0,neutral
1370529,Vitek,Sokol,2023-01-17,Первитин или аналоги.,1370468.0,neutral


Способ долгий, но возможно более точный. Берёт среднее из нескольких моделей и возвращает доминирующую эмоцию.

In [3]:
# Создание экземпляра класса EmotionRecognizer
emotion_recognizer = EmotionRecognizer()

# Пример DataFrame с текстом для анализа
data = {'text': ["I am feeling great today!", "I am so sad right now.", "I am extremely angry!"]}
df = pd.DataFrame(data)

# Применение функции recognize_average_emotion_from_multiple_models ко всем строкам DataFrame с average=True
df['emotion_average'] = emotion_recognizer.recognize_average_emotion_from_multiple_models(df, text_column='text', average=True)

# Применение функции recognize_average_emotion_from_multiple_models ко всем строкам DataFrame с average=False
df['emotion_max'] = emotion_recognizer.recognize_average_emotion_from_multiple_models(df, text_column='text', average=False)

# Вывод результата
print(df)

                        text emotion_average emotion_max
0  I am feeling great today!       happiness   happiness
1     I am so sad right now.         sadness   happiness
2      I am extremely angry!       happiness   happiness


In [4]:
df=df.head(100)

In [5]:
emotion_recognizer = EmotionRecognizer()

In [5]:
df

,name,last_name,date,text,post_id
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0
1370278,Irina,Vladimirovna,2023-01-16,"А кто знает,почему от этой школы шарахались,ка...",1370271.0
1370412,Alya,Dzhafarova,2023-01-17,"Интересно ,а как в нее можно попасть?Если в 3 ...",1370271.0
...,...,...,...,...,...
1370825,Nina,Khryakova,2023-01-19,Очень вкусная и отличная продукция все натурал...,1370426.0
1370547,Vika,Lee,2023-01-18,Не жрать что попало ))),1370468.0
1370537,Vyacheslav,Slivkin,2023-01-17,Есть в основной массе белки и сложные углеводы...,1370468.0
1370529,Vitek,Sokol,2023-01-17,Первитин или аналоги.,1370468.0


In [6]:
# Применение функции recognize_average_emotion_from_multiple_models ко всем строкам DataFrame с average=True
df['emotion_average'] = emotion_recognizer.recognize_average_emotion_from_multiple_models(df, 'text', average=True)

# Применение функции recognize_average_emotion_from_multiple_models ко всем строкам DataFrame с average=False
df['emotion_max'] = emotion_recognizer.recognize_average_emotion_from_multiple_models(df, 'text', average=False)

Processing models:   0%|          | 0/4 [00:00<?, ?it/s]

Processing models:  50%|█████     | 2/4 [00:22<00:24, 12.50s/it]

: 

In [ ]:
df

,name,last_name,date,text,post_id,emotion_average,emotion_max
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0,neutral,happiness
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0,neutral,happiness
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0,neutral,happiness
1370278,Irina,Vladimirovna,2023-01-16,"А кто знает,почему от этой школы шарахались,ка...",1370271.0,enthusiasm,happiness
1370412,Alya,Dzhafarova,2023-01-17,"Интересно ,а как в нее можно попасть?Если в 3 ...",1370271.0,enthusiasm,happiness
...,...,...,...,...,...,...,...
1370825,Nina,Khryakova,2023-01-19,Очень вкусная и отличная продукция все натурал...,1370426.0,happiness,happiness
1370547,Vika,Lee,2023-01-18,Не жрать что попало ))),1370468.0,happiness,happiness
1370537,Vyacheslav,Slivkin,2023-01-17,Есть в основной массе белки и сложные углеводы...,1370468.0,neutral,happiness
1370529,Vitek,Sokol,2023-01-17,Первитин или аналоги.,1370468.0,neutral,happiness


In [3]:
#Используются все 4 модели и считается среднее
df = df.head(4)
recognizer = EmotionRecognizer()
df['emotion'] = df['text'].apply(lambda text: recognizer.recognize_average_emotion_from_multiple_models(text, average = False))

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [4]:
df

,name,last_name,date,text,post_id,emotion
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0,neutral
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0,neutral
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0,neutral
1370278,Irina,Vladimirovna,2023-01-16,"А кто знает,почему от этой школы шарахались,ка...",1370271.0,enthusiasm


In [6]:
#Используются 1-4 модели, задаётся ввод вручную
from aniemore.models import HuggingFaceModel
models = [
    HuggingFaceModel.Text.Bert_Tiny,
    HuggingFaceModel.Text.Bert_Base
]
recognizer = EmotionRecognizer()

df = df.head(3)
df['emotion'] = df['text'].apply(lambda text: recognizer.recognize_average_emotion_from_multiple_models(text, models))

In [7]:
df

,name,last_name,date,text,post_id,emotion
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0,neutral
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0,neutral
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0,neutral
